# Lab 2

## Bayesian Classifier and Feature Importance

**Note:** Exercises can be autograded and count towards your lab and assignment score. Problems are graded for participation.

In [1]:
from pathlib import Path
home = str(Path.home()) # all other paths are relative to this path. change to something else if this is not the case on your system

In [2]:
#### NO NEED TO EDIT ####
from pathlib import Path
home = str(Path.home()) # all other paths are relative to this path. change to something else if this is not the case on your system
REPO = f"{home}/csc-466-student"
NOTEBOOK = "Final"

%load_ext autoreload
%autoreload 2

from importlib import import_module
helper = import_module(f'{NOTEBOOK}_helper')

#### NO NEED TO EDIT ####

For this lab, we are going to first implement an empirical naive bayesian classifier, then implement feature importance measures and apply it to a dataset, and finally, we will examine the affect of modifying the priors.

For developing this lab, we can use famous Titanic Kaggle dataset. Description of the data is found https://www.kaggle.com/c/titanic/data.

In [3]:
import pandas as pd
import numpy as np
cs = pd.read_csv(
    f"csgo_games.csv"
)
cs.head()

,match_date,team_1,team_2,t1_points,t2_points,t1_world_rank,t2_world_rank,t1_h2h_win_perc,t2_h2h_win_perc,winner,...,t2_player5_dpr,t2_player5_spr,t2_player5_opk_ratio,t2_player5_opk_rating,t2_player5_wins_perc_after_fk,t2_player5_fk_perc_in_wins,t2_player5_multikill_perc,t2_player5_rating_at_least_one_perc,t2_player5_is_sniper,t2_player5_clutch_win_perc
0,2016-12-18,CLOUD9,HELLRAISERS,13,16,9,20,0.500000,0.500000,t2,...,0.63,0.10,1.05,0.92,0.733,0.104,0.147018,0.528,False,0.666667
1,2016-12-18,IMMORTALS,G2,17,19,13,10,0.500000,0.500000,t2,...,0.69,0.09,0.85,0.96,0.739,0.147,0.135810,0.438,True,0.406250
2,2016-12-18,MOUSESPORTS,IMMORTALS,16,3,12,13,0.500000,0.500000,t1,...,0.67,0.07,0.79,0.94,0.723,0.129,0.148257,0.526,False,0.733333
3,2016-12-18,DIGNITAS,G2,16,9,6,10,0.416667,0.583333,t1,...,0.69,0.09,0.85,0.96,0.739,0.147,0.135810,0.438,True,0.406250
4,2016-12-18,OPTIC,HELLRAISERS,16,10,4,20,0.500000,0.500000,t1,...,0.63,0.10,1.05,0.92,0.733,0.104,0.147018,0.528,False,0.666667


We only need a few columns, and I will also perform some preprocessing for you:

In [4]:
allStats = ['rating','impact','kdr','dmr','kpr','apr','dpr','spr','opk_ratio','opk_rating','wins_perc_after_fk',
            'fk_perc_in_wins','multikill_perc','rating_at_least_one_perc','is_sniper','clutch_win_perc']
print('All unique stats available')
print(allStats)

allPlayers = helper.get_players()
print('A list of all players on both teams')
print(allPlayers)

p2All = helper.get_player_stats(1, 2)
print('All stats for player 2 of team 1')
print(p2All)

All unique stats available
['rating', 'impact', 'kdr', 'dmr', 'kpr', 'apr', 'dpr', 'spr', 'opk_ratio', 'opk_rating', 'wins_perc_after_fk', 'fk_perc_in_wins', 'multikill_perc', 'rating_at_least_one_perc', 'is_sniper', 'clutch_win_perc']
A list of all players on both teams
['t1_player1', 't1_player2', 't1_player3', 't1_player4', 't1_player5', 't2_player1', 't2_player2', 't2_player3', 't2_player4', 't2_player5']
All stats for player 2 of team 1
['t1_player2_rating', 't1_player2_impact', 't1_player2_kdr', 't1_player2_dmr', 't1_player2_kpr', 't1_player2_apr', 't1_player2_dpr', 't1_player2_spr', 't1_player2_opk_ratio', 't1_player2_opk_rating', 't1_player2_wins_perc_after_fk', 't1_player2_fk_perc_in_wins', 't1_player2_multikill_perc', 't1_player2_rating_at_least_one_perc', 't1_player2_is_sniper', 't1_player2_clutch_win_perc']


In [6]:
for player in allPlayers:
    cs[player + "_is_sniper"] = cs[player + "_is_sniper"].astype(int)

cs.winner.map(dict(t1=1, t2=0))


0       0.0
1       0.0
2       1.0
3       1.0
4       1.0
       ... 
3782    0.0
3783    0.0
3784    0.0
3785    0.0
3786    1.0
Name: winner, Length: 3787, dtype: float64

In [7]:
p1Stats = ['t1_player1_' + stat for stat in allStats]
print('All stats for player 1 on team 1')
print(p1Stats)

allRating = []
for player in allPlayers:
    allRating.append(player + '_rating')
print('Rating for each player on both teams')
print(allRating)

All stats for player 1 on team 1
['t1_player1_rating', 't1_player1_impact', 't1_player1_kdr', 't1_player1_dmr', 't1_player1_kpr', 't1_player1_apr', 't1_player1_dpr', 't1_player1_spr', 't1_player1_opk_ratio', 't1_player1_opk_rating', 't1_player1_wins_perc_after_fk', 't1_player1_fk_perc_in_wins', 't1_player1_multikill_perc', 't1_player1_rating_at_least_one_perc', 't1_player1_is_sniper', 't1_player1_clutch_win_perc']
Rating for each player on both teams
['t1_player1_rating', 't1_player2_rating', 't1_player3_rating', 't1_player4_rating', 't1_player5_rating', 't2_player1_rating', 't2_player2_rating', 't2_player3_rating', 't2_player4_rating', 't2_player5_rating']


In [8]:
allPlayersPerStat = helper.get_players_per_stat()
print('List of all players\' stats organized by each stat')
print(allPlayersPerStat)

List of all players' stats organized by each stat
{'rating': ['t1_player1_rating', 't1_player2_rating', 't1_player3_rating', 't1_player4_rating', 't1_player5_rating', 't2_player1_rating', 't2_player2_rating', 't2_player3_rating', 't2_player4_rating', 't2_player5_rating'], 'impact': ['t1_player1_impact', 't1_player2_impact', 't1_player3_impact', 't1_player4_impact', 't1_player5_impact', 't2_player1_impact', 't2_player2_impact', 't2_player3_impact', 't2_player4_impact', 't2_player5_impact'], 'kdr': ['t1_player1_kdr', 't1_player2_kdr', 't1_player3_kdr', 't1_player4_kdr', 't1_player5_kdr', 't2_player1_kdr', 't2_player2_kdr', 't2_player3_kdr', 't2_player4_kdr', 't2_player5_kdr'], 'dmr': ['t1_player1_dmr', 't1_player2_dmr', 't1_player3_dmr', 't1_player4_dmr', 't1_player5_dmr', 't2_player1_dmr', 't2_player2_dmr', 't2_player3_dmr', 't2_player4_dmr', 't2_player5_dmr'], 'kpr': ['t1_player1_kpr', 't1_player2_kpr', 't1_player3_kpr', 't1_player4_kpr', 't1_player5_kpr', 't2_player1_kpr', 't2_player2

In [9]:
p1 = cs.loc[:,p1All]
rating = cs.loc[:,allRating]
print('Before')
display(cs.head())
rating

NameError: name 'p1All' is not defined

In [10]:
allPlayersPerStat = helper.get_players_per_stat()
teams = helper.get_teams()
print(teams)

(['t1_player1', 't1_player2', 't1_player3', 't1_player4', 't1_player5'], ['t2_player1', 't2_player2', 't2_player3', 't2_player4', 't2_player5'])


In [ ]:
cs_norm = cs
teamStats = ['points', 'world_rank', 'h2h_win_perc']
for stat in teamStats:
    cs_norm[stat] = cs_norm['t1_' + stat] - cs_norm['t2_' + stat]
    cs_norm = cs_norm.drop('t1_' + stat, axis = 1)
    cs_norm = cs_norm.drop('t2_' + stat, axis = 1)

for stat in allPlayersPerStat.keys():
    t1 = [p + "_" + stat for p in teams[0]]
    t2 = [p + "_" + stat for p in teams[1]]
    cs_norm[stat] = (np.sum(cs_norm[t1], axis = 1) - np.sum(cs_norm[t2], axis = 1))
    cs_norm = cs_norm.drop(t1, axis = 1)
    cs_norm = cs_norm.drop(t2, axis = 1)

cs_norm = cs_norm.drop(['match_date', 'team_1', 'team_2'], axis = 1)
cs_norm

#### Exercise 1
Create a function to determine the prior probability of ALL the classes in ``y``. The result must be in the form of a Python dictionary such as ``priors = {'Survived=0': 0.4, 'Survived=1': 0.6}``.

In [6]:
survived_priors = helper.compute_priors(titanic_df['Survived'])
survived_priors


{'Survived=0': 0.6161616161616161, 'Survived=1': 0.3838383838383838}

In [7]:
helper.compute_priors(titanic_df['Age'])

{'Age=0': 0.06958473625140292,
 'Age=10': 0.11447811447811448,
 'Age=20': 0.44556677890011226,
 'Age=30': 0.18742985409652077,
 'Age=40': 0.09988776655443322,
 'Age=50': 0.05387205387205387,
 'Age=60': 0.02132435465768799,
 'Age=70': 0.006734006734006734,
 'Age=80': 0.001122334455667789}

In [8]:
y_example = titanic_df['Age']
y_example.name

'Age'

#### Exercise 2
Create a function to calculate the specific class conditional probability. Assume x and y are pd.Series objects. Assume xv and yv are specific values. This function should return $\Pr(x==xv|y==yv)$.



In [10]:
prob = helper.specific_class_conditional(titanic_df['Sex'],'female',titanic_df['Survived'],0)
prob

0.14754098360655737

In [11]:
!pytest -vv --diff-symbols {REPO}/tests/test_{NOTEBOOK}.py::test_exercise_2

============================= test session starts ==============================
platform linux -- Python 3.9.7, pytest-7.1.2, pluggy-1.0.0 -- /opt/tljh/user/bin/python3.9
cachedir: .pytest_cache
rootdir: /home/jupyter-waltz@calpoly.edu
plugins: clarity-1.0.1, anyio-3.5.0
collected 1 item                                                               

../csc-466-student/tests/test_Lab2.py::test_exercise_2 PASSED            [100%]

============================== 1 passed in 0.38s ===============================


#### Exercise 3
Now construct a dictionary based data structure that stores all possible class conditional probabilities (e.g., loop through all possible combinations of values). The keys in your dictionary should be of the form "pclass=1|survived=0". ``X`` is a ``pd.DataFrame`` object and ``y`` is a ``pd.Series`` object. You can retrieve the name of the series object ``y`` by accessing ``y.name``.

Aside: I know it might be a bit annoying to store the key of this dictionary as a string instead of as say a tuple of tuples, but I think the way this prints for folks learning this is reason enough to use strings in this instance.

In [9]:
X = titanic_df.drop("Survived",axis=1)
y = titanic_df["Survived"]
probs = helper.class_conditional(X,y)
probs


{'Pclass=3|Survived=0': 0.6775956284153005,
 'Pclass=3|Survived=1': 0.347953216374269,
 'Pclass=1|Survived=0': 0.14571948998178508,
 'Pclass=1|Survived=1': 0.39766081871345027,
 'Pclass=2|Survived=0': 0.1766848816029144,
 'Pclass=2|Survived=1': 0.2543859649122807,
 'Sex=male|Survived=0': 0.8524590163934426,
 'Sex=male|Survived=1': 0.31871345029239767,
 'Sex=female|Survived=0': 0.14754098360655737,
 'Sex=female|Survived=1': 0.6812865497076024,
 'Age=20|Survived=0': 0.48816029143898,
 'Age=20|Survived=1': 0.37719298245614036,
 'Age=30|Survived=0': 0.17122040072859745,
 'Age=30|Survived=1': 0.2134502923976608,
 'Age=50|Survived=0': 0.051001821493624776,
 'Age=50|Survived=1': 0.05847953216374269,
 'Age=0|Survived=0': 0.04371584699453552,
 'Age=0|Survived=1': 0.1111111111111111,
 'Age=10|Survived=0': 0.1111111111111111,
 'Age=10|Survived=1': 0.11988304093567251,
 'Age=40|Survived=0': 0.10018214936247723,
 'Age=40|Survived=1': 0.09941520467836257,
 'Age=60|Survived=0': 0.023679417122040074,


In [13]:
!pytest -vv --diff-symbols {REPO}/tests/test_{NOTEBOOK}.py::test_exercise_3

============================= test session starts ==============================
platform linux -- Python 3.9.7, pytest-7.1.2, pluggy-1.0.0 -- /opt/tljh/user/bin/python3.9
cachedir: .pytest_cache
rootdir: /home/jupyter-waltz@calpoly.edu
plugins: clarity-1.0.1, anyio-3.5.0
collected 1 item                                                               

../csc-466-student/tests/test_Lab2.py::test_exercise_3 PASSED            [100%]

============================== 1 passed in 0.39s ===============================


#### Exercise 4
Now you are ready to calculate the posterior probabilities for a given sample. Write and test the following function that returns a dictionary where the keys are of the form "Survived=0|Pclass=1,Sex=male,Age=60". Make sure you return 0.5 if the specific combination of values does not exist. ``probs`` and ``priors`` are defined the same as above. ``x`` is a pd.Series object that represents a specific example/sample from our dataset.

In [14]:
probs = helper.class_conditional(X,y)
priors = helper.compute_priors(y)
x = titanic_df.drop("Survived",axis=1).loc[2]
x

Pclass         3
Sex       female
Age           20
Name: 2, dtype: object

In [15]:
post_probs = helper.posteriors(probs,priors,x)
post_probs

{'Survived=0|Pclass=3,Sex=female,Age=20': 0.46699312907215196,
 'Survived=1|Pclass=3,Sex=female,Age=20': 0.533006870927848}

Here is one more test you should check. Your code should return 50/50 if given a value that is not in the empirical distribution.

In [16]:
x = titanic_df.drop("Survived",axis=1).loc[2]
x = x.copy()
x['Age'] = 200
x

Pclass         3
Sex       female
Age          200
Name: 2, dtype: object

In [17]:
post_probs = helper.posteriors(probs,priors,x)
post_probs

{'Survived=0|Pclass=3,Sex=female,Age=200': 0.5,
 'Survived=1|Pclass=3,Sex=female,Age=200': 0.5}

In [18]:
!pytest -vv --diff-symbols {REPO}/tests/test_{NOTEBOOK}.py::test_exercise_4

============================= test session starts ==============================
platform linux -- Python 3.9.7, pytest-7.1.2, pluggy-1.0.0 -- /opt/tljh/user/bin/python3.9
cachedir: .pytest_cache
rootdir: /home/jupyter-waltz@calpoly.edu
plugins: clarity-1.0.1, anyio-3.5.0
collected 1 item                                                               

../csc-466-student/tests/test_Lab2.py::test_exercise_4 PASSED            [100%]

============================== 1 passed in 0.39s ===============================


#### Exercise 5
All this is great, but how would you evaluate how we are doing? Create a function called train_test_split that splits our dataframe into a training and testing dataset. To make sure this is done randomly, I have inserted a shuffle into the code for you. The ``test_frac`` is the fraction of the dataset that will be held out for testing.

In [8]:
import numpy as np
np.random.seed(2)
Xtrain,ytrain,Xtest,ytest=helper.train_test_split(X,y)
print('Xtrain')
display(Xtrain.head())
print('ytrain')
display(ytrain.head())
print('Xtest')
display(Xtest.head())
print('ytest')
display(ytest.head())

NameError: name 'X' is not defined

In [20]:
!pytest -vv --diff-symbols {REPO}/tests/test_{NOTEBOOK}.py::test_exercise_5

============================= test session starts ==============================
platform linux -- Python 3.9.7, pytest-7.1.2, pluggy-1.0.0 -- /opt/tljh/user/bin/python3.9
cachedir: .pytest_cache
rootdir: /home/jupyter-waltz@calpoly.edu
plugins: clarity-1.0.1, anyio-3.5.0
collected 1 item                                                               

../csc-466-student/tests/test_Lab2.py::test_exercise_5 PASSED            [100%]

============================== 1 passed in 0.39s ===============================


#### Exercise 6
For this exercise, create a training dataset of size 50% and then using your solutions to previous exercises, find the prediction accuracy for the test dataset. **NOTE:** When/if there is a tie in the posterior probabilities, your code should predict that the passenger passed away.

In [21]:

Xtrain.drop_duplicates().sort_values('Age')
ytest

716    1
154    0
843    0
24     0
294    0
      ..
534    0
584    0
493    0
527    0
168    0
Name: Survived, Length: 445, dtype: int64

In [22]:
# 0.7820224719101123
np.random.seed(0)
Xtrain,ytrain,Xtest,ytest=helper.train_test_split(X,y)
x = Xtest.iloc[0]
x

Pclass       2
Sex       male
Age         30
Name: 213, dtype: object

In [23]:
accuracy = helper.exercise_6(Xtrain,ytrain,Xtest,ytest)
accuracy

0.7820224719101123

That's not bad!

In [27]:
!pytest -vv --diff-symbols {REPO}/tests/test_{NOTEBOOK}.py::test_exercise_6

============================= test session starts ==============================
platform linux -- Python 3.9.7, pytest-7.1.2, pluggy-1.0.0 -- /opt/tljh/user/bin/python3.9
cachedir: .pytest_cache
rootdir: /home/jupyter-waltz@calpoly.edu
plugins: clarity-1.0.1, anyio-3.5.0
collected 1 item                                                               

../csc-466-student/tests/test_Lab2.py::test_exercise_6 PASSED            [100%]

============================== 1 passed in 0.44s ===============================


In [25]:
accuracy = sum(ytest == 0)/ytest.size
accuracy

0.597752808988764

#### Problem 2:
Is that better than guessing all passengers died? What is the accuracy on the test set if you guessed all passengers died?

Yes, it is better than predicting that all passengers died. The accuracy of the test set if we guess that all passengers die is (shown above) ~.6 and the accuracy from our prediction model is 0.782

**Upload your solution/answer here:** https://canvas.calpoly.edu/courses/89325/assignments/594334

**Before proceeding**, make sure you have read the required reading of section 8.5.1 of [this book by Christoph Molnar](https://christophm.github.io/interpretable-ml-book/feature-importance.html). 

#### Excercise 7. Create a function that returns the test based feature importance for our Bayesian classifier.

In [10]:
np.random.seed(0)
Xtrain,ytrain,Xtest,ytest=helper.train_test_split(X,y)
importances = helper.exercise_7(Xtrain,ytrain,Xtest,ytest)
importances

{'Pclass': 0.02966292134831472,
 'Sex': 0.19550561797752808,
 'Age': 0.016853932584269593}

In [36]:
!pytest -vv --diff-symbols {REPO}/tests/test_{NOTEBOOK}.py::test_exercise_7

============================= test session starts ==============================
platform linux -- Python 3.9.7, pytest-7.1.2, pluggy-1.0.0 -- /opt/tljh/user/bin/python3.9
cachedir: .pytest_cache
rootdir: /home/jupyter-waltz@calpoly.edu
plugins: clarity-1.0.1, anyio-3.5.0
collected 1 item                                                               

../csc-466-student/tests/test_Lab2.py::test_exercise_7 PASSED            [100%]

============================== 1 passed in 1.93s ===============================


#### Excercise 8. Create a function that returns the train based feature importance for our Bayesian classifier.

In [37]:
np.random.seed(0)
Xtrain,ytrain,Xtest,ytest=helper.train_test_split(X,y)
importances = helper.exercise_8(Xtrain,ytrain,Xtest,ytest)
importances

{'Pclass': -0.00955056179775271,
 'Sex': 0.10471910112359528,
 'Age': -0.006516853932584277}

In [38]:
!pytest -vv --diff-symbols {REPO}/tests/test_{NOTEBOOK}.py::test_exercise_8

============================= test session starts ==============================
platform linux -- Python 3.9.7, pytest-7.1.2, pluggy-1.0.0 -- /opt/tljh/user/bin/python3.9
cachedir: .pytest_cache
rootdir: /home/jupyter-waltz@calpoly.edu
plugins: clarity-1.0.1, anyio-3.5.0
collected 1 item                                                               

../csc-466-student/tests/test_Lab2.py::test_exercise_8 PASSED            [100%]

============================== 1 passed in 3.38s ===============================


#### Problem 3. After you implement this, what is the most important feature? 

The most important feature is Sex. This is demonstrated by sex being the dominant classifier (~10x more important) in both the test based feature importance and the train based feature importance.

**Upload your solution/answer here:** https://canvas.calpoly.edu/courses/89325/assignments/594335

#### Problem 4: What are the differences between the two sets of importances?

The difference between these two sets are where the importance is measured. The test set shows the actual accuracy of our features compared to random values, while the training set primarily shows if any of our data has been overfitted to the training set.

**Upload your solution/answer here:** https://canvas.calpoly.edu/courses/89325/assignments/594336

#### Problem 5: What does a negative value for the importances mean? Consider that it is a very small importance, so what does it say about these features? Consider that question in the context of what you are permuting (training or testing).

A negative value of importance means that creating a random sample of values for that feature creates less error than if the original values were to be preserved. The fact that it is near zero means that in the first place, these values are of little importance and might as well be permuted. In the context of permuting both training and testing, it means that the data in the training set is a bad fit for the test set.

**Upload your solution/answer here:** https://canvas.calpoly.edu/courses/89325/assignments/594337

Another thing to add to your brain, are there any correlations between features?

In [22]:
Xtrain.corr()

,Pclass,Age
Pclass,1.000000,-0.395434
Age,-0.395434,1.000000


In [23]:
Xtest.corr()

,Pclass,Age
Pclass,1.000000,-0.281029
Age,-0.281029,1.000000


In [24]:
# Good job!
# Don't forget to push with ./submit.sh

#### Having trouble with the test cases and the autograder?

You can always load up the answers for the autograder. The autograder runs your code and compares your answer to the expected answer. I manually review your code, so there is no need to hide this from you.

In [24]:
import joblib
answers = joblib.load(f"{REPO}/tests/answers_Lab2.joblib")
answers.keys()

dict_keys(['exercise_1', 'exercise_2', 'exercise_3', 'exercise_4', 'exercise_5', 'exercise_6', 'exercise_7', 'exercise_8'])

In [25]:
answers['exercise_6']

0.7820224719101123